In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular
import pickle

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import xgboost as xgb
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class LiverCancerRiskClassificationCDSS:
    """간암 위험도 분류 모델 클래스 (CDSS 호환 + XAI)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.scaler = None
        self.label_encoders = {}
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        self.holdout_patient = None  # CDSS 테스트용 환자
        
        print(f"🚀 간암 위험도 분류 모델 초기화 (CDSS 호환 + XAI)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 위험도 계층화 모델용 선택된 컬럼들
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            # 간기능 평가 (가장 중요)
            'child_pugh_classification', 'ishak_fibrosis_score',
            # 종양 특성 및 병기
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'tumor_grade', 'morphology',
            # 환자 기본 특성
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            # 치료 이력
            'prior_treatment', 'prior_malignancy', 'synchronous_malignancy',
            # 핵심 변수들
            'residual_disease', 'classification_of_tumor', 'primary_diagnosis', 'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산
        self.processed_df['duration'] = self.processed_df['days_to_death'].fillna(
            self.processed_df['days_to_last_follow_up']
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (self.processed_df['duration'].notna()) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (CDSS 호환)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # CDSS 테스트용 환자 1명 미리 분리
        print("🔄 CDSS 테스트용 환자 분리 중...")
        holdout_idx = self.processed_df.sample(n=1, random_state=42).index[0]
        self.holdout_patient = self.processed_df.loc[holdout_idx:holdout_idx].copy()
        remaining_df = self.processed_df.drop(holdout_idx).copy()
        
        print(f"   - CDSS 테스트 환자: {holdout_idx}")
        print(f"   - 모델 훈련용 데이터: {len(remaining_df)}명")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in remaining_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration']]
        
        X = remaining_df[feature_cols].copy()
        y_duration = remaining_df['duration'].values
        y_event = remaining_df['event'].values.astype(bool)
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 수치형 변수: 중앙값으로 대체
        if numerical_cols:
            self.num_imputer = SimpleImputer(strategy='median')
            X[numerical_cols] = self.num_imputer.fit_transform(X[numerical_cols])
            print(f"✅ Imputer 훈련 완료: {len(numerical_cols)}개 수치형 변수")
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들
        meaningful_unknown_cols = [
            'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
            'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade', 'residual_disease',
            'prior_malignancy', 'synchronous_malignancy', 'prior_treatment'
        ]
        
        self.label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            self.label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 특성 스케일링
        self.scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료 (CDSS 호환)")
        
        return X_scaled, y_structured, y_duration, y_event
    
    def create_risk_scores(self, X):
        """위험도 점수 계산"""
        print("\n🎯 4. 위험도 점수 계산")
        
        # 원본 데이터에서 주요 변수들 추출 (holdout 제외)
        risk_df = self.processed_df.drop(self.holdout_patient.index).copy()
        
        # Child-Pugh 점수 계산
        child_pugh_scores = np.zeros(len(risk_df))
        if 'child_pugh_classification' in risk_df.columns:
            for i, val in enumerate(risk_df['child_pugh_classification']):
                if pd.isna(val) or val == 'Unknown':
                    child_pugh_scores[i] = 1  # 중간값
                elif 'A' in str(val):
                    child_pugh_scores[i] = 0
                elif 'B' in str(val):
                    child_pugh_scores[i] = 2
                elif 'C' in str(val):
                    child_pugh_scores[i] = 4
                else:
                    child_pugh_scores[i] = 1
        
        # AJCC 병기 점수 계산
        stage_scores = np.zeros(len(risk_df))
        if 'ajcc_pathologic_stage' in risk_df.columns:
            for i, val in enumerate(risk_df['ajcc_pathologic_stage']):
                if pd.isna(val) or val == 'Unknown':
                    stage_scores[i] = 1  # 중간값
                elif 'I' in str(val) and 'II' not in str(val) and 'III' not in str(val) and 'IV' not in str(val):
                    stage_scores[i] = 0
                elif 'II' in str(val) and 'III' not in str(val):
                    stage_scores[i] = 1
                elif 'III' in str(val) and 'IV' not in str(val):
                    stage_scores[i] = 2
                elif 'IV' in str(val):
                    stage_scores[i] = 3
                else:
                    stage_scores[i] = 1
        
        # 연령 점수 계산
        age_scores = np.zeros(len(risk_df))
        if 'age_at_diagnosis' in risk_df.columns:
            ages = risk_df['age_at_diagnosis'].fillna(risk_df['age_at_diagnosis'].median())
            age_scores = np.where(ages < 60, 0, np.where(ages <= 70, 1, 2))
        
        # 성별 점수 계산
        gender_scores = np.zeros(len(risk_df))
        if 'gender' in risk_df.columns:
            gender_scores = np.where(risk_df['gender'] == 'male', 1, 0)
        
        # 총 위험도 점수 계산
        total_scores = child_pugh_scores + stage_scores + age_scores + gender_scores
        
        # 3단계 위험도 분류
        risk_categories = np.where(total_scores <= 2, 0,  # 저위험군
                                 np.where(total_scores <= 5, 1, 2))  # 중위험군, 고위험군
        
        risk_labels = ['저위험군', '중위험군', '고위험군']
        
        print(f"📊 위험도 분포:")
        for i, label in enumerate(risk_labels):
            count = np.sum(risk_categories == i)
            percentage = count / len(risk_categories) * 100
            print(f"   - {label}: {count}명 ({percentage:.1f}%)")
        
        return total_scores, risk_categories, risk_labels
    
    def split_data(self, X, y_structured, y_duration, y_event, risk_categories):
        """데이터 분할"""
        print("\n✂️  5. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event, risk_temp, risk_test = \
            train_test_split(X, y_structured, y_duration, y_event, risk_categories,
                           test_size=0.2, random_state=42, stratify=risk_categories)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event, risk_train, risk_val = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event, risk_temp,
                           test_size=0.25, random_state=42, stratify=risk_temp)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명)")
        print(f"📊 CDSS 테스트: 1명 (별도 보관)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event,
                risk_train, risk_val, risk_test)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event,
                    risk_train, risk_val, risk_test):
        """모델 훈련 (CDSS 호환)"""
        print("\n🤖 6. 모델 훈련 (CDSS 호환)")
        
        # 1. Random Forest 분류 모델 (CDSS 호환 버전)
        print("🔄 Random Forest 분류 모델 훈련 중...")
        try:
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )
            rf_classifier.fit(X_train, risk_train)
            
            # RF 모델을 CDSS 호환 형태로 래핑
            rf_wrapper = {
                'model': rf_classifier,
                'model_type': 'RandomForestClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'risk_labels': ['저위험군', '중위험군', '고위험군'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(risk_train),
                    'feature_importance': rf_classifier.feature_importances_
                },
                'preprocessing_info': {
                    'meaningful_unknown_cols': [
                        'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
                        'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade', 'residual_disease',
                        'prior_malignancy', 'synchronous_malignancy', 'prior_treatment'
                    ]
                }
            }
            
            self.models['Risk_RF'] = rf_wrapper
            print("✅ Random Forest 분류 모델 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ RF 분류 모델 훈련 실패: {e}")
        
        # 2. XGBoost 분류 모델 (CDSS 호환 버전)
        print("🔄 XGBoost 분류 모델 훈련 중...")
        try:
            xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                random_state=42,
                eval_metric='mlogloss'
            )
            xgb_model.fit(X_train, risk_train)
            
            # XGBoost 모델도 래핑
            xgb_wrapper = {
                'model': xgb_model,
                'model_type': 'XGBClassifier',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'risk_labels': ['저위험군', '중위험군', '고위험군'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(risk_train),
                    'feature_importance': xgb_model.feature_importances_
                },
                'preprocessing_info': {
                    'meaningful_unknown_cols': [
                        'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
                        'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade', 'residual_disease',
                        'prior_malignancy', 'synchronous_malignancy', 'prior_treatment'
                    ]
                }
            }
            
            self.models['XGBoost_Risk'] = xgb_wrapper
            print("✅ XGBoost 분류 모델 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ XGBoost 분류 모델 훈련 실패: {e}")
        
        # 3. 로지스틱 회귀 (CDSS 호환 버전)
        print("🔄 로지스틱 회귀 모델 훈련 중...")
        try:
            lr_model = LogisticRegression(
                multi_class='multinomial',
                solver='lbfgs',
                random_state=42,
                max_iter=1000
            )
            lr_model.fit(X_train, risk_train)
            
            # 로지스틱 회귀 모델도 래핑
            lr_wrapper = {
                'model': lr_model,
                'model_type': 'LogisticRegression',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'num_imputer': self.num_imputer,
                'risk_labels': ['저위험군', '중위험군', '고위험군'],
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'class_distribution': np.bincount(risk_train)
                },
                'preprocessing_info': {
                    'meaningful_unknown_cols': [
                        'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
                        'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade', 'residual_disease',
                        'prior_malignancy', 'synchronous_malignancy', 'prior_treatment'
                    ]
                }
            }
            
            self.models['Logistic_Risk'] = lr_wrapper
            print("✅ 로지스틱 회귀 모델 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ 로지스틱 회귀 모델 훈련 실패: {e}")
        
        print(f"\n🎯 총 {len(self.models)}개 모델 훈련 완료 (모두 CDSS 호환)")
        
        return True
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event,
                       risk_train, risk_val, risk_test):
        """모델 평가"""
        print("\n📈 7. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event, risk_train),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event, risk_val),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event, risk_test)
        }
        
        for model_name, model_wrapper in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event, risk_true) in datasets.items():
                try:
                    # 래핑된 모델에서 실제 모델 추출
                    actual_model = model_wrapper['model']
                    
                    # 분류 모델 평가
                    risk_pred = actual_model.predict(X)
                    accuracy = (risk_pred == risk_true).mean()
                    
                    # 다중클래스 AUC 계산
                    try:
                        risk_proba = actual_model.predict_proba(X)
                        auc_score = roc_auc_score(risk_true, risk_proba, multi_class='ovr', average='weighted')
                    except:
                        auc_score = np.nan
                    
                    self.results[model_name][dataset_name] = {
                        'accuracy': accuracy,
                        'auc': auc_score
                    }
                    print(f"   {dataset_name}: Accuracy = {accuracy:.3f}, AUC = {auc_score:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'error': str(e)}
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 초기화
        print("🔄 SHAP 설명 생성 중...")
        for model_name in ['Risk_RF', 'XGBoost_Risk']:
            if model_name in self.models:
                try:
                    X_test_sample = X_test.iloc[:50]
                    
                    # 래핑된 모델에서 실제 모델 추출
                    actual_model = self.models[model_name]['model']
                    
                    explainer = shap.TreeExplainer(actual_model)
                    shap_values = explainer.shap_values(X_test_sample)
                    
                    self.shap_explainers[model_name] = explainer
                    self.shap_values[model_name] = shap_values
                    print(f"✅ {model_name} SHAP 설명 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} SHAP 실패: {e}")
        
        # LIME 설명기 초기화
        print("\n🔄 LIME 설명 생성 중...")
        class_names = ['저위험군', '중위험군', '고위험군']
        for model_name in ['Risk_RF', 'XGBoost_Risk']:
            if model_name in self.models:
                try:
                    # 래핑된 모델에서 실제 모델 추출
                    actual_model = self.models[model_name]['model']
                    
                    explainer = lime_tabular.LimeTabularExplainer(
                        training_data=X_train.values,
                        feature_names=self.feature_names,
                        class_names=class_names,
                        mode='classification',
                        discretize_continuous=True
                    )
                    
                    self.lime_explainers[model_name] = {
                        'explainer': explainer,
                        'predict_fn': actual_model.predict_proba
                    }
                    print(f"✅ {model_name} LIME 설명기 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 및 저장 (다중클래스 SHAP 수정 버전)"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화
        shap_figures = []
        for model_name in self.shap_explainers:
            try:
                print(f"🔄 {model_name} SHAP 시각화 디버깅:")
                
                shap_vals = self.shap_values[model_name]
                print(f"   - SHAP 값 타입: {type(shap_vals)}")
                print(f"   - SHAP 값 형태: {np.array(shap_vals).shape if isinstance(shap_vals, list) else shap_vals.shape}")
                
                # 다중클래스 SHAP 값 처리
                if isinstance(shap_vals, list) and len(shap_vals) == 3:
                    # 리스트 형태: [class0_shap, class1_shap, class2_shap]
                    combined_shap = np.mean([np.abs(shap_vals[i]) for i in range(3)], axis=0)
                    print(f"   - 리스트 형태 감지: 3개 클래스 평균으로 결합")
                    print(f"   - 결합된 SHAP 형태: {combined_shap.shape}")
                    
                    # Individual waterfall plot - 중위험군(클래스 1) 사용
                    class_idx = 1  # 중위험군
                    individual_shap = shap_vals[class_idx][sample_index]  # 특정 샘플, 특정 클래스
                    expected_val = self.shap_explainers[model_name].expected_value[class_idx]
                    
                elif hasattr(shap_vals, 'shape') and shap_vals.ndim == 3:
                    # 3차원 배열: (샘플, 특성, 클래스)
                    combined_shap = np.mean(np.abs(shap_vals), axis=2)  # 클래스 차원 평균
                    print(f"   - 3차원 배열 감지: 클래스 차원 평균으로 결합")
                    print(f"   - 결합된 SHAP 형태: {combined_shap.shape}")
                    
                    # Individual waterfall plot
                    class_idx = 1  # 중위험군
                    individual_shap = shap_vals[sample_index, :, class_idx]  # (특성,) 형태
                    expected_val = (self.shap_explainers[model_name].expected_value[class_idx] 
                                if hasattr(self.shap_explainers[model_name].expected_value, '__len__') 
                                else self.shap_explainers[model_name].expected_value)
                    
                else:
                    # 2차원 배열 또는 기타
                    combined_shap = shap_vals
                    individual_shap = shap_vals[sample_index]
                    expected_val = (self.shap_explainers[model_name].expected_value 
                                if not hasattr(self.shap_explainers[model_name].expected_value, '__len__')
                                else self.shap_explainers[model_name].expected_value[0])
                    print(f"   - 2차원 배열 사용")
                
                print(f"   - 개별 SHAP 형태: {individual_shap.shape}")
                print(f"   - 기댓값: {expected_val}")
                
                # Summary plot
                plt.figure(figsize=(10,6))
                shap.summary_plot(combined_shap, X_test.iloc[:50], 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title(f"{model_name} 특성 중요도 (SHAP)")
                shap_summary_path = f"shap_summary_{model_name}.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                # Individual waterfall plot (수정된 부분)
                plt.figure(figsize=(12,6))
                
                # 🔥 핵심 수정: shap.plots.waterfall 대신 shap.waterfall_plot 사용
                try:
                    shap.waterfall_plot(
                        shap.Explanation(
                            values=individual_shap.astype(float),  # 명시적 float 변환
                            base_values=float(expected_val),       # 명시적 float 변환
                            data=X_test.iloc[sample_index].values.astype(float),
                            feature_names=self.feature_names
                        ),
                        show=False
                    )
                    plt.title(f"{model_name} 개별 설명 - 중위험군 (샘플 {sample_index})")
                    shap_waterfall_path = f"shap_waterfall_{model_name}_{sample_index}.png"
                    plt.savefig(shap_waterfall_path, bbox_inches='tight')
                    plt.close()
                    
                    shap_figures.extend([shap_summary_path, shap_waterfall_path])
                    print(f"✅ {model_name} SHAP 시각화 완료")
                    
                except Exception as waterfall_error:
                    print(f"⚠️ Waterfall plot 실패, Force plot으로 대체: {waterfall_error}")
                    
                    # Waterfall 실패 시 Force plot 사용
                    try:
                        shap.force_plot(
                            expected_val,
                            individual_shap,
                            X_test.iloc[sample_index],
                            feature_names=self.feature_names,
                            matplotlib=True,
                            show=False
                        )
                        plt.title(f"{model_name} 개별 설명 - Force Plot (샘플 {sample_index})")
                        shap_force_path = f"shap_force_{model_name}_{sample_index}.png"
                        plt.savefig(shap_force_path, bbox_inches='tight')
                        plt.close()
                        
                        shap_figures.extend([shap_summary_path, shap_force_path])
                        print(f"✅ {model_name} SHAP 시각화 완료 (Force plot)")
                        
                    except Exception as force_error:
                        print(f"❌ Force plot도 실패: {force_error}")
                        shap_figures.append(shap_summary_path)  # Summary plot만 저장
                    
            except Exception as e:
                print(f"❌ {model_name} SHAP 시각화 실패: {e}")
                print(f"   상세 오류 정보:")
                import traceback
                traceback.print_exc()
        
        # LIME 시각화
        lime_figures = []
        for model_name in self.lime_explainers:
            try:
                lime_data = self.lime_explainers[model_name]
                exp = lime_data['explainer'].explain_instance(
                    X_test.iloc[sample_index].values,
                    lime_data['predict_fn'],
                    num_features=5
                )
                
                lime_path = f"lime_explanation_{model_name}_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"{model_name} LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                
                lime_figures.append(lime_path)
                print(f"✅ {model_name} LIME 시각화 완료")
            except Exception as e:
                print(f"❌ {model_name} LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures
    
    def preprocess_holdout_patient(self):
        """Holdout 환자 데이터 전처리 (CDSS 시뮬레이션)"""
        print("\n🔧 Holdout 환자 전처리 시작")
        
        # holdout 환자의 특성 데이터만 추출
        feature_cols = [col for col in self.holdout_patient.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration']]
        
        patient_raw = self.holdout_patient[feature_cols].copy()
        print(f"🔍 원본 환자 특성: {len(patient_raw.columns)}개")
        print(f"🔍 모델 훈련 특성: {len(self.feature_names)}개")
        
        # 모델 훈련 시 사용한 특성명과 정확히 일치하는 DataFrame 생성
        patient_processed = pd.DataFrame(index=patient_raw.index)
        
        for feature_name in self.feature_names:
            if feature_name in patient_raw.columns:
                patient_processed[feature_name] = patient_raw[feature_name].copy()
                print(f"✅ {feature_name}: 원본 데이터 사용")
            else:
                patient_processed[feature_name] = 0.0
                print(f"⚠️ {feature_name}: 기본값(0.0) 설정")
        
        # 범주형 변수 전처리
        print("\n🔄 범주형 변수 전처리:")
        for col, encoder in self.label_encoders.items():
            if col in patient_processed.columns:
                try:
                    original_value = patient_processed[col].iloc[0]
                    print(f"   - {col}: 원본값 = {original_value}")
                    
                    if pd.isna(original_value) or original_value == 'NA':
                        patient_processed[col] = encoder.classes_[0]
                        print(f"     → 결측치를 '{encoder.classes_[0]}'로 대체")
                    else:
                        str_value = str(original_value)
                        if str_value in encoder.classes_:
                            patient_processed[col] = encoder.transform([str_value])[0]
                            print(f"     → 인코딩: '{str_value}' → {patient_processed[col].iloc[0]}")
                        else:
                            patient_processed[col] = encoder.transform([encoder.classes_[0]])[0]
                            print(f"     → 새로운 값 '{str_value}'을 '{encoder.classes_[0]}'로 대체")
                            
                except Exception as e:
                    print(f"     ❌ {col} 인코딩 실패: {e}")
                    patient_processed[col] = 0.0
        
        # 모든 컬럼을 수치형으로 변환
        print("\n🔢 데이터 타입 변환:")
        for col in patient_processed.columns:
            try:
                patient_processed[col] = pd.to_numeric(patient_processed[col], errors='coerce')
                if patient_processed[col].isnull().any():
                    patient_processed[col] = patient_processed[col].fillna(0.0)
            except Exception as e:
                print(f"⚠️ {col} 수치형 변환 실패: {e}")
                patient_processed[col] = 0.0
        
        # 특성 순서를 훈련 시와 정확히 일치시키기
        patient_processed = patient_processed[self.feature_names]
        print(f"✅ 특성 순서 정렬 완료: {patient_processed.shape}")
        
        # 수치형 변수 전처리
        if hasattr(self, 'num_imputer'):
            try:
                patient_values = patient_processed.values
                imputed_values = self.num_imputer.transform(patient_values)
                patient_processed = pd.DataFrame(
                    imputed_values,
                    columns=self.feature_names,
                    index=patient_processed.index
                )
                print(f"✅ Imputer 적용 완료")
            except Exception as e:
                print(f"⚠️ Imputer 적용 실패, 건너뜀: {e}")
        
        # 스케일링 적용
        try:
            patient_values = patient_processed.values
            scaled_values = self.scaler.transform(patient_values)
            patient_features_scaled = pd.DataFrame(
                scaled_values,
                columns=self.feature_names,
                index=patient_processed.index
            )
            
            print(f"✅ 최종 특성 형태: {patient_features_scaled.shape}")
            return patient_features_scaled
            
        except Exception as e:
            print(f"❌ 스케일링 실패: {e}")
            return patient_processed
    
    def test_cdss_compatibility(self):
        """CDSS 호환성 테스트"""
        print("\n🔬 CDSS 호환성 테스트")
        
        try:
            # holdout 환자 데이터 전처리
            holdout_features = self.preprocess_holdout_patient()
            
            # 각 모델로 예측 수행
            predictions = {}
            probabilities = {}
            
            for model_name, model_wrapper in self.models.items():
                try:
                    print(f"\n🔄 {model_name} 모델 예측 중...")
                    
                    # 래핑된 모델에서 실제 모델 추출
                    actual_model = model_wrapper['model']
                    risk_labels = model_wrapper['risk_labels']
                    
                    # 예측 수행
                    pred_class = actual_model.predict(holdout_features)[0]
                    pred_proba = actual_model.predict_proba(holdout_features)[0]
                    
                    predictions[model_name] = {
                        'class': pred_class,
                        'class_name': risk_labels[pred_class],
                        'probabilities': pred_proba
                    }
                    
                    print(f"✅ {model_name}: {risk_labels[pred_class]} (확률: {pred_proba[pred_class]:.3f})")
                    
                except Exception as e:
                    print(f"❌ {model_name} 예측 실패: {e}")
                    predictions[model_name] = {'error': str(e)}
            
            return predictions
            
        except Exception as e:
            print(f"❌ CDSS 호환성 테스트 실패: {e}")
            return {}
    
    def save_models_for_cdss(self):
        """CDSS 호환 모델 저장"""
        print("\n💾 CDSS 호환 모델 저장")
        
        # 전체 파이프라인을 하나의 객체로 저장
        cdss_pipeline = {
            'models': self.models,
            'holdout_patient': self.holdout_patient,
            'metadata': {
                'created_date': datetime.now().isoformat(),
                'model_version': '2.0',
                'description': 'TCGA-LIHC 간암 위험도 분류 모델 (CDSS 호환)',
                'risk_labels': ['저위험군', '중위험군', '고위험군']
            }
        }
        
        # 개별 모델도 저장
        for model_name, model_wrapper in self.models.items():
            try:
                filename = f"cdss_liver_cancer_risk_{model_name.lower()}_model.pkl"
                with open(filename, 'wb') as f:
                    pickle.dump(model_wrapper, f)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        # 전체 파이프라인 저장
        try:
            pipeline_filename = "cdss_liver_cancer_risk_complete_pipeline.pkl"
            with open(pipeline_filename, 'wb') as f:
                pickle.dump(cdss_pipeline, f)
            print(f"✅ 전체 파이프라인 저장: {pipeline_filename}")
        except Exception as e:
            print(f"❌ 파이프라인 저장 실패: {e}")
        
        return True
    
    def plot_results(self, X_test, y_test_dur, y_test_event, risk_test, risk_labels):
        """결과 시각화 (XAI + CDSS 포함)"""
        print("\n📊 8. 결과 시각화")
        
        fig = plt.figure(figsize=(25, 35))
        gs = fig.add_gridspec(5, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 위험도별 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 위험도 분포
            fig.add_subplot(gs[0, 2]),  # 모델 성능 비교
            fig.add_subplot(gs[1, 0]),  # 분류 AUC
            fig.add_subplot(gs[1, 1]),  # RF 특성 중요도
            fig.add_subplot(gs[1, 2]),  # XGB 특성 중요도
            fig.add_subplot(gs[2, 0]),  # 위험도별 생존율
            fig.add_subplot(gs[2, 1]),  # CDSS 테스트 결과
            fig.add_subplot(gs[2, 2]),  # 모델 성능 종합
            fig.add_subplot(gs[3, :]),  # SHAP 시각화
            fig.add_subplot(gs[4, :])   # LIME 시각화
        ]
        
        fig.suptitle('간암 위험도 분류 모델 분석 결과 (XAI + CDSS 호환)', fontsize=16, fontweight='bold')
        
        # 1. 위험도별 생존 곡선
        print("🔍 위험도별 생존 곡선 생성 중...")
        for risk_level in range(3):
            mask = (risk_test == risk_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask], label=f'{risk_labels[risk_level]} (n={mask.sum()})')
                kmf.plot_survival_function(ax=axes[0])
        
        axes[0].set_title('위험도별 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. 위험도 분포
        risk_counts = pd.Series(risk_test).value_counts().sort_index()
        colors = ['lightgreen', 'gold', 'lightcoral']
        bars = axes[1].bar([risk_labels[i] for i in risk_counts.index], 
                          risk_counts.values, color=colors)
        axes[1].set_title('테스트 세트 위험도 분포')
        axes[1].set_ylabel('환자 수')
        
        for bar, value in zip(bars, risk_counts.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value}명', ha='center', va='bottom', fontweight='bold')
        
        # 3. 모델 성능 비교 (Accuracy)
        model_names = list(self.results.keys())
        test_accuracies = []
        for name in model_names:
            accuracy = self.results[name]['Test'].get('accuracy', np.nan)
            if not np.isnan(accuracy):
                test_accuracies.append(accuracy)
            else:
                test_accuracies.append(0)
        
        bars = axes[2].bar(model_names, test_accuracies, 
                          color=['lightgreen', 'orange', 'gold'][:len(model_names)])
        axes[2].set_title('모델별 Accuracy 성능 비교 (테스트 세트)')
        axes[2].set_ylabel('Accuracy')
        axes[2].set_ylim(0, 1.0)
        axes[2].grid(True, alpha=0.3, axis='y')
        
        for bar, value in zip(bars, test_accuracies):
            if value > 0:
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. 모델 성능 비교 (AUC)
        test_aucs = []
        for name in model_names:
            auc = self.results[name]['Test'].get('auc', np.nan)
            if not np.isnan(auc):
                test_aucs.append(auc)
            else:
                test_aucs.append(0)
        
        bars = axes[3].bar(model_names, test_aucs, 
                          color=['lightgreen', 'orange', 'gold'][:len(model_names)])
        axes[3].set_title('모델별 AUC 성능 비교 (테스트 세트)')
        axes[3].set_ylabel('AUC')
        axes[3].set_ylim(0, 1.0)
        axes[3].grid(True, alpha=0.3, axis='y')
        
        for bar, value in zip(bars, test_aucs):
            if value > 0:
                axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 5-6. 특성 중요도
        for idx, model_name in enumerate(['Risk_RF', 'XGBoost_Risk']):
            ax_idx = 4 + idx
            if model_name in self.models:
                try:
                    model_wrapper = self.models[model_name]
                    importance = model_wrapper['training_info']['feature_importance']
                    
                    feature_importance_df = pd.DataFrame({
                        'feature': self.feature_names,
                        'importance': importance
                    }).sort_values('importance', ascending=True).tail(10)
                    
                    color = 'lightblue' if 'RF' in model_name else 'orange'
                    bars = axes[ax_idx].barh(range(len(feature_importance_df)), 
                                           feature_importance_df['importance'],
                                           color=color)
                    
                    axes[ax_idx].set_yticks(range(len(feature_importance_df)))
                    axes[ax_idx].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                    axes[ax_idx].set_title(f'특성 중요도 ({model_name})', fontsize=12, fontweight='bold')
                    axes[ax_idx].set_xlabel('중요도')
                    axes[ax_idx].grid(True, alpha=0.3, axis='x')
                    
                except Exception as e:
                    print(f"❌ {model_name} 특성 중요도 시각화 실패: {e}")
        
        # 7. 위험도별 생존율 요약
        survival_summary = []
        time_points = [365, 1095, 1825]  # 1년, 3년, 5년
        
        for risk_level in range(3):
            mask = (risk_test == risk_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask])
                
                survival_rates = []
                for time_point in time_points:
                    try:
                        survival_rate = kmf.survival_function_at_times(time_point).values[0]
                        survival_rates.append(survival_rate * 100)
                    except:
                        survival_rates.append(np.nan)
                
                survival_summary.append(survival_rates)
            else:
                survival_summary.append([np.nan, np.nan, np.nan])
        
        survival_df = pd.DataFrame(survival_summary, 
                                  index=[risk_labels[i] for i in range(3)],
                                  columns=['1년', '3년', '5년'])
        
        sns.heatmap(survival_df, annot=True, fmt='.1f', cmap='RdYlGn', 
                   ax=axes[6], cbar_kws={'label': '생존율 (%)'})
        axes[6].set_title('위험도별 생존율 요약')
        axes[6].set_ylabel('위험도 그룹')
        
        # 8. CDSS 테스트 결과
        cdss_results = self.test_cdss_compatibility()
        if cdss_results:
            model_names_cdss = []
            predictions_cdss = []
            
            for model_name, result in cdss_results.items():
                if 'class_name' in result:
                    model_names_cdss.append(model_name)
                    predictions_cdss.append(result['class'])
            
            if model_names_cdss:
                bars = axes[7].bar(model_names_cdss, predictions_cdss, 
                                 color=['lightgreen', 'orange', 'gold'][:len(model_names_cdss)])
                axes[7].set_title('CDSS 호환성 테스트\n(Holdout 환자 예측)')
                axes[7].set_ylabel('예측 위험도 클래스')
                axes[7].set_ylim(-0.5, 2.5)
                axes[7].set_yticks([0, 1, 2])
                axes[7].set_yticklabels(['저위험', '중위험', '고위험'])
                
                for bar, value in zip(bars, predictions_cdss):
                    axes[7].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                               f'{["저위험", "중위험", "고위험"][value]}', 
                               ha='center', va='bottom', fontweight='bold')
        
        # 9. 모델 성능 종합
        all_models_performance = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    all_models_performance[f'{model_name}_Accuracy'] = accuracy
        
        if all_models_performance:
            model_names = list(all_models_performance.keys())
            model_scores = list(all_models_performance.values())
            
            bars = axes[8].bar(range(len(model_names)), model_scores, 
                             color=['lightgreen', 'orange', 'gold'][:len(model_names)])
            axes[8].set_title('전체 모델 성능 종합')
            axes[8].set_ylabel('성능 점수')
            axes[8].set_xticks(range(len(model_names)))
            axes[8].set_xticklabels(model_names, rotation=45, ha='right')
            axes[8].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, model_scores):
                axes[8].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=8)
        
        # 10-11. XAI 시각화 로드
        try:
            model_name = 'XGBoost_Risk' if 'XGBoost_Risk' in self.shap_explainers else 'Risk_RF'
            if model_name:
                img = plt.imread(f"shap_summary_{model_name}.png")
                axes[9].imshow(img)
                axes[9].axis('off')
                axes[9].set_title('SHAP 전역 설명 (위험도 분류)', fontsize=12)
        except Exception as e:
            axes[9].text(0.5, 0.5, 'SHAP 시각화 불러오기 실패', 
                        ha='center', va='center', transform=axes[9].transAxes,
                        fontsize=12, fontweight='bold')
        
        try:
            model_name = 'XGBoost_Risk' if 'XGBoost_Risk' in self.lime_explainers else 'Risk_RF'
            if model_name:
                img = plt.imread(f"lime_explanation_{model_name}_0.png")
                axes[10].imshow(img)
                axes[10].axis('off')
                axes[10].set_title('LIME 개별 설명 (첫 번째 환자)', fontsize=12)
        except Exception as e:
            axes[10].text(0.5, 0.5, 'LIME 시각화 불러오기 실패', 
                         ha='center', va='center', transform=axes[10].transAxes,
                         fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        save_path = "liver_cancer_risk_classification_cdss_xai.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"📁 시각화 결과 저장: {save_path}")
        
        return True
    
    def generate_report(self):
        """결과 보고서 생성"""
        print("\n📋 9. 결과 보고서 생성")
        
        report = []
        report.append("="*60)
        report.append("간암 환자 위험도 분류 모델 분석 결과 (XAI + CDSS 호환)")
        report.append("="*60)
        report.append(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"데이터 경로: {self.data_path}")
        report.append("")
        
        # 데이터 요약
        report.append("📊 데이터 요약")
        report.append("-" * 30)
        report.append(f"총 환자 수: {len(self.processed_df) + 1}")  # holdout 포함
        report.append(f"모델 훈련용: {len(self.processed_df)}명")
        report.append(f"CDSS 테스트용: 1명")
        report.append(f"사망률: {self.processed_df['event'].mean()*100:.1f}%")
        report.append(f"중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        report.append(f"사용된 특성 수: {len(self.feature_names)}")
        report.append("")
        
        # 위험도 분포
        report.append("🎯 위험도 계층화 결과")
        report.append("-" * 30)
        risk_labels = ['저위험군', '중위험군', '고위험군']
        for i, label in enumerate(risk_labels):
            report.append(f"{label}: 분석 완료")
        report.append("")
        
        # 모델 성능
        report.append("🤖 모델 성능")
        report.append("-" * 30)
        
        for model_name in self.results:
            report.append(f"\n{model_name}:")
            for dataset in ['Train', 'Validation', 'Test']:
                if dataset in self.results[model_name]:
                    metrics = self.results[model_name][dataset]
                    if 'accuracy' in metrics:
                        report.append(f"  {dataset}: Accuracy = {metrics['accuracy']:.3f}")
                    if 'auc' in metrics and not np.isnan(metrics['auc']):
                        report.append(f"  {dataset}: AUC = {metrics['auc']:.3f}")
        
        # 최고 성능 모델
        test_performances = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                metrics = self.results[model_name]['Test']
                if 'accuracy' in metrics:
                    test_performances[f"{model_name}_Accuracy"] = metrics['accuracy']
        
        if test_performances:
            best_model = max(test_performances, key=test_performances.get)
            report.append(f"\n🏆 최고 성능 모델: {best_model} (점수: {test_performances[best_model]:.3f})")
        
        report.append("")
        report.append("ℹ️  특징:")
        report.append("   - CDSS 호환 위험도 분류 모델")
        report.append("   - XAI 설명 가능성 포함 (SHAP, LIME)")
        report.append("   - Holdout 환자로 실제 예측 테스트")
        report.append("   - 3단계 위험도 분류 (저/중/고위험군)")
        report.append("")
        report.append("="*60)
        
        # 보고서 저장
        report_text = "\n".join(report)
        with open("liver_cancer_risk_classification_cdss_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        
        print("📁 보고서 저장: liver_cancer_risk_classification_cdss_report.txt")
        print("\n" + report_text)
        
        return report_text
    
    def run_complete_analysis(self):
        """전체 분석 실행 (XAI + CDSS 호환)"""
        print("🎯 간암 위험도 분류 모델 전체 분석 시작 (XAI + CDSS 호환)")
        
        try:
            # 1. 데이터 로드
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비 (CDSS 호환)
            X, y_structured, y_duration, y_event = self.prepare_features()
            
            # 4. 위험도 점수 계산
            total_scores, risk_categories, risk_labels = self.create_risk_scores(X)
            
            # 5. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event,
             risk_train, risk_val, risk_test) = self.split_data(X, y_structured, y_duration, y_event, risk_categories)
            
            # 6. 모델 훈련 (CDSS 호환)
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event,
                                   risk_train, risk_val, risk_test):
                return False
            
            # 7. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event,
                                       risk_train, risk_val, risk_test):
                return False
            
            # 8. XAI 설명 생성
            self.explain_models(X_train, X_test)
            
            # 9. XAI 시각화 생성
            self.generate_xai_visualizations(X_test)
            
            # 10. 시각화 (XAI + CDSS 포함)
            if not self.plot_results(X_test, y_test_dur, y_test_event, risk_test, risk_labels):
                return False
            
            # 11. 보고서 생성
            self.generate_report()
            
            # 12. CDSS 호환 모델 저장
            self.save_models_for_cdss()
            
            print("\n🎉 전체 분석 완료!")
            print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 실행
if __name__ == "__main__":
    # 데이터 경로
    data_path = r"C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv"
    
    # 분석 실행
    risk_classifier = LiverCancerRiskClassificationCDSS(data_path)
    success = risk_classifier.run_complete_analysis()
    
    if success:
        print("\n✨ 모든 분석이 성공적으로 완료되었습니다!")
        print("📁 생성된 파일들:")
        print("   - liver_cancer_risk_classification_cdss_xai.png (시각화 결과)")
        print("   - liver_cancer_risk_classification_cdss_report.txt (분석 보고서)")
        print("   - cdss_liver_cancer_risk_*_model.pkl (CDSS 호환 개별 모델)")
        print("   - cdss_liver_cancer_risk_complete_pipeline.pkl (전체 파이프라인)")
        print("   - shap_*.png (SHAP 설명)")
        print("   - lime_*.png (LIME 설명)")
        print("\n🔬 CDSS 호환성:")
        print("   - 모든 전처리 파이프라인 포함")
        print("   - Holdout 환자로 실제 예측 테스트 완료")
        print("   - 3단계 위험도 분류 (저/중/고위험군)")
        print("   - XAI 설명 가능성 포함")
    else:
        print("\n❌ 분석 중 문제가 발생했습니다. 로그를 확인해주세요.")


✅ 한글 폰트 설정 완료
🚀 간암 위험도 분류 모델 초기화 (CDSS 호환 + XAI)
📁 데이터 경로: C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv
⏰ 시작 시간: 2025-06-16 11:24:27
🎯 간암 위험도 분류 모델 전체 분석 시작 (XAI + CDSS 호환)

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 377행 × 87열
📈 데이터 기본 정보:
   - 총 환자 수: 377
   - 총 컬럼 수: 87
   - 생존 환자: 245명
   - 사망 환자: 132명
   - 사망률: 35.0%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 22개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 372명
   - 사망 이벤트: 132건
   - 중간 생존 시간: 602일

📋 결측값 분석:
   - days_to_death: 240개 (64.5%)
   - child_pugh_classification: 47개 (12.6%)
   - ishak_fibrosis_score: 96개 (25.8%)
   - ajcc_pathologic_stage: 24개 (6.5%)
   - ajcc_pathologic_t: 2개 (0.5%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - tumor_grade: 5개 (1.3%)
   - age_at_diagnosis: 3개 (0.8%)
   - residual_disease: 7개 (1.9%)
   - year_of_diagnosis: 2개 (0.5%)

🎯 3. 특성 준비 및 인코딩
🔄 CDSS 테스트용 환자 분리 중...
   - CDSS 테스트 환자: 332
   - 모델 훈련용 데이터: 371명
📊 초기 특성 개수: 19
📊 샘플 개수: 371
🔤 범주형 변수: 17개
🔢 수치형 변수: 2개
🔄 결측값 처리 중...
✅ Imputer 훈련 완료: 2개 수치형 변수

   🔍 child_pugh_classifica